

***
<div class="alert alert-block alert-info">

# Multimedia fugacity models
# – LEVEL II –
Authors: Antonia Praetorius (a.praetorius@uva.nl) and Marianne Seijo (m.x.seijo@uva.nl) <br><br>
</div>

Copyright: 
_The present codes are a suport for teaching and stay under the copyright of the authors_<br>
Use of the software constitutes your agreement to abide by the terms and conditions set out in the license agreement.
Models are provided "as is" with no guarantees, as described in the License agreements. Users are urged to read the License agreement provided with each model prior to installing or using the software. And, although these models are provided as "freeware", in fairness to the developers, the funding agencies and partners, and to all users, they are copyright.    
    

Databases within the models are not intended as an authoritative information source and all physical chemical properties should be verified by the user. Users should be especially careful about the temperature dependence of physical chemical properties.    
    


***

<div class="alert alert-block alert-success">
    
## Learning Objectives:
The goal of this suite of models is to have a first interaction with environmental fate modelling using multimedia models and understand the connections between physico-chemical properties of chemicals and their environmental fate.
    
</div>    

***

<div class="alert alert-block alert-success">
    

## Lesson Reminders: 
    

### Level II: Environmental partitioning with loss processes    
    
    
A Level II simulation is designed to model the following: 
    
- An open environment system
- A chemical emitted at a constant rate E (in mol/h or kg/h)
- Degradation and advection are included (expressed by D-values)
- Interphase equilibrium is still assumed
- The overall system is at steady state
    
With Level II, we introduce **advection** and **reactions** of the chemicals: It grants the first estimate of environmental persistence, an important property of the overall fate of a chemical in the environment. The Level II model remains an evaluative model. By introducing loss processes Level II is more releasitic than Level I, but interphase equilibrium is still assumed. In a Level II model the chemical is emitted continously at a fixed rate E and reaches a steady- steady condition (with equilibrium between the phases) where input and output rates in the overall system are equal (see Figure below). It is irrelevant into which compartment the chemical is emitted, as it is instantaneously distributed throughout the whole system (due to the interphase equilibrium assumption).

![LEVEL2_Figure1](img/LEVEL2_Figure1.png)
    
    
A Level II calculation is useful for establishing:
        
=> The general features of a new or existing chemical's behaviour.    
=> The likely media a chemical will tend to persist.    
    
    

The loss process (degradation and advection) are expressed in D-values following the fugacity approach. D-values are calcualted from the fugacity capacities Z of a given phase and for advection from advective flow rates G (m3/h) and for degradation from the degradation rates k of a given environmental reaction (e.g. biodegradation):
    
    D_adv = G * Z
    D_react = V * Z * k


       
   

</div>    
    
    
<br><br>



***
## Level II Modelisation step by step: 



We will work with the following example, inspired from :    
Mackay, D. Multimedia Environmental Models: The Fugacity Approach, Second Edition; CRC Press, 2001,     
Chapter 6.


![LEVEL2_Figure2](img/LEVEL2_Figure2.png)
<br>




***

# Python Libraries, Versions:    
## Tested and Validated with Following Libraries:

Python 3 version 3.9 </br>
Pandas version 1.5.1</br>
Ipywidgets:  8.0.2 </br>

## Import Libraries needed and versions available:

In [ ]:
import sys

restart = False
    
try:
    # pandas is used to read excel file
    import pandas as pd
    print("pandas="+str(pd.__version__))
except ImportError:
    print ('pandas is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install pandas
    restart = True
 
    
try:
    # plotly is used to create map and data on it
    import plotly
    print("plotly="+str(plotly.__version__))
except ImportError:
    print ('plotly is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install plotly
    restart = True  
       
    
import math   


try:
    # ipywidgets is used to create map and data on it
    import ipywidgets
    print("ipywidgets="+str(ipywidgets.__version__))
except ImportError:
    print ('ipywidgets is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install ipywidgets
    restart = True  


***
***
# Input Parameters

## Constants

In [ ]:
R = 8.314  # gas constant (Pa.m3/mol K)

## Environmental input:

In [ ]:
# Surface (m2): ------------------------
S_Air      = 1.0e11
S_Soil     = 10.0e10
S_Water    = 9.0e10
S_Sediment = 9.0e10

# Depth (m): --------------------------
depth_Air      = 1.0e3
depth_Soil     = 0.2
depth_Water    = 20.0
depth_Sediment = 0.05

# Volume (m3): ------------------------
V_Air      = S_Air * depth_Air
V_Soil     = S_Soil * depth_Soil
V_Water    = S_Water * depth_Water
V_Sediment = S_Sediment * depth_Sediment


# Density (kg.m-3): --------------------
density_Air      = 1.185413
density_Soil     = 2400.0
density_Water    = 1000.0
density_Sediment = 2400.0

density_Aerosol        = 2000.0
density_Susp_Particles = 1500.0
density_Fish           = 1000.0

# Volume Fractions: ---------------------------
Vf_Susp_Particles_In_Water = 5.0e-3
Vf_Fish_In_Water           = 1.0e-3   
Vf_Aerosol_In_Air         = 2.0e-6

# Lipid In Fish: (L.kg-1)---------------------------
L_Lipid_In_Fish           = 0.05  

# Fraction Organic Carbon for solid (g.g-1) ---------------
foc_Suspended_Particles = 0.15
foc_Soil                = 0.1
foc_BottomSediment      = 0.05

# For Level II:
# Flow Rates (m3.h-1) ---------------
AirFlowR = 2.0e12 
WaterFlowR = 1.0e8
SedimentFlowR = 1.14
FishFlowR = 35.0

AerosolFlowR = AirFlowR * Vf_Aerosol_In_Air
Susp_ParticulesFlowR = WaterFlowR * Vf_Susp_Particles_In_Water 



## Chemicals Properties Inputs

In [ ]:
import math

Name                = "1,4-Dichlorobenzene"
Molar_Mass          = 147.00      # M  (g.mol-1)
Data_Temperature    = 25.0        # T  (°C)
Melting_Point       = 53.5        # Tm (°C)
Vapor_Pressure      = 170.0       # P (Pa) 
Solubility_In_Water = 73.5        # S (g.m-3) 
logKOW              = 3.4         

# Henry's law constant:  
# take fromcChemicals Properties Inputs if available, otherwise estimate as stated here
# Comment the unneeded line (by adding # in the beginning of the unneeded line), uncomment the one needed

# Henry_Constant = 37.0
Henry_Constant = Molar_Mass * Vapor_Pressure / Solubility_In_Water #(Pa·m3.mol-1)


# Level 2: Emission rate:
EmissionRate      = 1600.0         # (kg.h-1) Emission Rate=> Level II




# Half Lives (hours) ------
# time required for C to decrease to half of CO
t05_Air = 23.1
t05_Water = 9.9
t05_Soil = 23.0
t05_Sediment = 33.0
t05_Aerosol = 23.1
t05_SuspParticles = 23.105
t05_Fish = 23.105


# Level 2: Rate constants of reactions (1/hour):
ln2 = math.log(2.0)
if (t05_Air  != 0.0):
    k_Air = ln2 / t05_Air 
else: 
    k_Air  = 0.0

if (t05_Water != 0.0):
    k_Water = ln2 / t05_Water
else: 
    k_Water = 0.0

if (t05_Soil != 0.0):
    k_Soil = ln2 / t05_Soil
else: 
    k_Soil = 0.0


if (t05_Sediment != 0.0):
    k_Sediment = ln2 / t05_Sediment
else: 
    k_Sediment = 0.0

if (t05_Aerosol != 0.0):
    k_Aerosol = ln2 / t05_Aerosol 
else: 
    k_Aerosol = 0.0

    
if (t05_SuspParticles != 0.0):
    k_SuspParticles = ln2 / t05_SuspParticles
else: 
    k_SuspParticles = 0.0

    
if (t05_Fish != 0.0):
    k_Fish = ln2 / t05_Fish 
else: 
    k_Fish = 0.0


***
***

# Calculations
***
## Partition Coefficients Calculations

As in Level I:

| Compartments                |     Partition Coefficient (L.kg-1)                    
|:-----------------------------|:------------------------------------------------------------
| **(1) Air-Water**              | ${K_{AW} = \frac{H}{(R*T)}}$                          
| **(2) Water-Sediment**         | K$_{SedW}$ = K$_{OC}$ *foc$_{BottomSediment}$ 
| **(3) Water-Soil**             | K$_{SoilW}$ = K$_{OC}$ *foc$_{Soil}$ 
| **(4) Air-Soil**              | ${K_{SoilA}=\frac{K_{SoilW}*\rho_{Soil}}{K_{AW}*\rho_{Water}}}$  
| **(5) Fish-Water**             | ${K_{FW} = K_{OW} * L}$
| **(6) Susp.Particles-Water**    | K$_{Susp.Part.W}$ = K$_{OC}$ *foc$_{Susp. Part.}$
| **(7) Aerosol-Air**            | ${K_{AeA} = \frac{6.10^{6}} { P_L}}$





With:    
    
**H**, the Henry's law constant ($Pa.m^3.mol^{-1}$)     
**R**, the ideal gas constant = 8.314 $Pa.m^3.mol^{-1}.K$    
**T**, the Temperature in Kelvin (K)

**K$_{OC}$**, the Organic Carbon-Water Partition Coefficient     
**f$_{OC}$**, the Fraction Organic Carbon for solid (g.g-1)    

**$\rho_{Soil}$** = density of the solid phase S (kg.L$^{-1}$)    
**$\rho_{Water}$** = density of the biota on the media W (kg.L$^{-1}$)    

K$_{OW}$, the Oil-Water Partition Coefficient   
L : fish lipids content

$P_L$: Subcooled Liquid Vapor Pressure.   
&emsp;For Tm the melting point temperature, T the temperature and P the Gas Pressure, 
if T>Tm: ${P_L = P.exp^{[6.79.((Tm/T)-1)]}}$ 


#### Organic Carbon-Water Partition Coefficient : 
The K$_{OC}$ depend on the nature of the media (soil composition, etc.) and the chemical component: <br>
cf https://setac.onlinelibrary.wiley.com/doi/pdf/10.1897/01-362 <br>
Two equations are proposed here: <br>
- **=> <ins> Karickhoff method</ins> : $\boxed{K_{OC_K} = 0.41 * K_{OW}}$** <br>
Karickhoff SW. 1984. Organic pollutant sorption in aquatic sys-tems.J Hydraul Eng Am Soc Civil Eng110:707–735  <br>
cf: https://doi.org/10.1061/(ASCE)0733-9429(1984)110:6(707) <br><br>

- **=> <ins> Seth et al. method</ins> : $\boxed{K_{OC_S} = 0.35 * K_{OW}}$** <br>
Seth  R,  Mackay  D,  Muncke  J.  1999.  Estimating  the  organiccarbon  partition  coefficient and  its  variability for hydrophobicchemicals.Environ Sci Technol33:2390–2394.  <br>
cf: https://pubs.acs.org/doi/full/10.1021/es980893j <br>

In [ ]:
# Air-Water Partition Coefficient (KAW) ------------------------------------------ 
KAW = Henry_Constant / (R *  (Data_Temperature + 273.15)) 

# ******* Calculate from logKOW ***********- 
KOW = pow(10.0,logKOW)         
                         
# Organic Carbon-Water Partition Coefficient (KOC): ------------------------------ 
# The KOC depend on the nature of soil and the chemical components: cf https://setac.onlinelibrary.wiley.com/doi/pdf/10.1897/01-362
# Two equations are proposed here. Other may be added depending on the chemical componant.
KOC_K = 0.41 * KOW # (L.kg-1) Karickhoff method. Karickhoff SW. 1984. Organic pollutant sorption in aquatic sys-tems.J Hydraul Eng Am Soc Civil Eng110:707–735                                                       
KOC_S = 0.35 * KOW # (L.kg-1) Seth et al. method hydrophobic organics. Seth  R,  Mackay  D,  Muncke  J.  1999.  Estimating  the  organiccarbon  partition  coefficient and  its  variability for hydrophobicchemicals.Environ Sci Technol33:2390–2394.                                        

#Selected KOC for the chemical:
KOC = KOC_K

# Fish-Water Partition Coefficient (KFW): --------------------------------
KFW = L_Lipid_In_Fish * KOW                         
                         
# Suspended Particle-Water Partition Coefficient (KSPW): --------------------------------
KSPW = KOC * foc_Suspended_Particles

# Soil-Water Partition Coefficient (KSW): --------------------------------
KSW = KOC * foc_Soil

# Sediment-Water Partition Coefficient (KSedWat): --------------------------------
KSedW = KOC * foc_BottomSediment 


# Soil-Air Partition Coefficient (KSA): --------------------------------
KSA = KSW * density_Soil / (KAW * density_Water )


# Aerosol-Air Partition Coefficient (KAeA): --------------------------------
KAeA = 0.0

if (Data_Temperature < Melting_Point):
    tempRatio = (Melting_Point+273.15)/(Data_Temperature+273.15)-1
    PL = Vapor_Pressure * math.exp(6.79*tempRatio)
    KAeA = 6e6/PL


dataPCoef = {'Partition Coefficient':  ['K_Air-Water', 
                                   'K_Soil-Air', 
                                   'K_Soil-Water',
                                   'K_Sediment-Water',
                                   'K_Fish-Water',
                                   'K_Suspended Particle-Water',
                                   'K_Aerosol-Air',     
                                  ],
               'Value': [KAW , KSA, KSW, KSedW, KFW, KSPW, KAeA]
        }

dfPC = pd.DataFrame(dataPCoef)
print('Partition coefficients calculated:')
display(dfPC)


***
## Z and VZ

As in level I:    
Different way to compute Z with partition coefficients previously calculated.     
We choose the following:     

| Compartments                 |     Definition of Z (mol.m-3.Pa)                 |      
|:-----------------------------|:----------------------|
| **Air**                      | ${K_{AW}/H}$ =1/(R*T)              |          
| **Water**                    | ${1/H}$        |
| **Soil**                     | ${K_{SW}/H}$          |
| **Sediments**                | ${K_{SedW}/H}$        |
| **Fish**                     | ${K_{FishW}/H}$       |
| **Susp.Particles**           | ${K_{SPW}/H}$         |
| **Aerosol**                  | ${K_{AeA}*Z_{Air}}$   |    


In [ ]:
# Z ----------------
ZAir = KAW/Henry_Constant
ZWater = 1.0/Henry_Constant
ZSoil = KSW/Henry_Constant
ZSediment = KSedW/Henry_Constant
ZFish = KFW/Henry_Constant
ZSuspendedParticles = KSPW/Henry_Constant
ZAerosol = KAeA*ZAir

# VZ ----------------
VZAir = ZAir * V_Air * (1.0 - Vf_Aerosol_In_Air)
VZWater = ZWater * V_Water*(1.0 - Vf_Fish_In_Water - Vf_Susp_Particles_In_Water)
VZSoil = ZSoil * V_Soil
VZSediment = ZSediment * V_Sediment
VZFish = ZFish * V_Water * Vf_Fish_In_Water
VZSuspendedParticles = ZSuspendedParticles * V_Water*Vf_Susp_Particles_In_Water
VZAerosol = ZAerosol* V_Air * Vf_Aerosol_In_Air

TotalVZ = VZAir + VZWater + VZSoil + VZSediment + VZFish + VZSuspendedParticles + VZAerosol
print("Total VZ (mol.Pa-1) = ", TotalVZ)



***
## D from Losses Advection process:   

### D$_{mediaXAdv}$ = Z$_{mediaX}$  * FlowRates$_{mediaX}$  

In [ ]:
# D ------
DAir_Adv = ZAir * AirFlowR
DWater_Adv = ZWater * WaterFlowR
DSediment_Adv = ZSediment * SedimentFlowR
DSuspParticles_Adv = ZSuspendedParticles * Susp_ParticulesFlowR
DAerosol_Adv = ZAerosol * AerosolFlowR
DFish_Adv = ZFish * FishFlowR



## D from Losses Reaction process:
### D$_{mediaXReac}$ = V$_{mediaX}$  * Z$_{mediaX}$  * k$_{mediaX}$  

In [ ]:
# D ------
DAir_Reac = VZAir * k_Air
DWater_Reac = VZWater * k_Water
DSoil_Reac = VZSoil * k_Soil
DSediment_Reac = VZSediment * k_Sediment
DSuspParticles_Reac = VZSuspendedParticles * k_Soil
DAerosol_Reac = VZAerosol * k_Aerosol
DFish_Reac = VZFish * k_Fish


## D  Total (Advection+Reactions)

In [ ]:
# D ------
D_Adv = DAir_Adv + DWater_Adv + DSediment_Adv + DSuspParticles_Adv + DAerosol_Adv + DFish_Adv
D_Reac = DAir_Reac + DSoil_Reac + DWater_Reac + DSediment_Reac + DSuspParticles_Reac + DAerosol_Reac + DFish_Reac
D_Tot = D_Adv + D_Reac


***
## Fugacity in Level II


### Fugacity (in Pa) = Total mol of Emission Rate of Chemical in Environment / D$_{Tot}$

In [ ]:
Fugacity = (1000.0  * EmissionRate / Molar_Mass) / D_Tot
print("Fugacity = : {0:e}".format(Fugacity), "Pa")


***
## Concentrations and Amounts on each media:

### C$_{mediaX}$ (in g.m-3 )= Molar_Mass * Fugacity * Z$_{mediaX}$     
### Amount$_{mediaX}$ (in kg )= Molar_Mass * Fugacity * V$_{mediaX}$ Z$_{mediaX}$ 

In [ ]:

# Concentrations (g.m-3) ----------------
CAir = Molar_Mass * Fugacity * ZAir
CWater = Molar_Mass * Fugacity * ZWater
CSoil = Molar_Mass * Fugacity * ZSoil
CSediment = Molar_Mass * Fugacity * ZSediment
CFish = Molar_Mass * Fugacity * ZFish
CSuspendedParticles = Molar_Mass * Fugacity * ZSuspendedParticles
CAerosol = Molar_Mass * Fugacity * ZAerosol


# Amounts (kg) ----------------
mm = Molar_Mass / 1000.0
AAir = mm * Fugacity * VZAir
AWater = mm * Fugacity * VZWater
ASoil = mm * Fugacity * VZSoil 
ASediment = mm * Fugacity * VZSediment
AFish = mm * Fugacity * VZFish 
ASuspendedParticles = mm * Fugacity * VZSuspendedParticles
AAerosol = mm * Fugacity * VZAerosol

total_Mass = AAir + AWater + ASoil + ASediment + AFish + ASuspendedParticles + AAerosol


# Percentage ----------------
pAir = 100.0 * AAir / total_Mass
pWater = 100.0 * AWater / total_Mass
pSoil = 100.0 * ASoil / total_Mass
pSediment = 100.0 * ASediment / total_Mass
pFish = 100.0 * AFish / total_Mass
pSuspendedParticles = 100.0 * ASuspendedParticles / total_Mass
pAerosol = 100.0 * AAerosol / total_Mass



## Losses by advective processes


**G** : Flow Rate (m3.h-1)    
**D** : Transport parameters (mol.Pa-1.h-1)= G.Z     
**Residence Time** =VolumeFraction*Volume Media / G     

**Loss Rates (mol.h-1)** = Fugacity * D_${Adv}$ 


In [ ]:
# Residence Time (days)----------------

if (AirFlowR!= 0.0):
    RTAir = V_Air * (1.0 - Vf_Aerosol_In_Air) / (24.0 * AirFlowR)
    RTAerosol_In_Air = V_Air * Vf_Aerosol_In_Air / (24.0 * AirFlowR)
else:
    RTAir = 'infinite'
    RTAerosol_In_Air = 'infinite'

if (WaterFlowR!= 0.0):
    RTWater = V_Water * (1.0 - Vf_Fish_In_Water - Vf_Susp_Particles_In_Water) / (24.0 * WaterFlowR)
    RTFish = V_Water * Vf_Fish_In_Water / (24.0 * WaterFlowR)
    RTSusp_Particles_In_Water = V_Water * Vf_Susp_Particles_In_Water / (24.0 * WaterFlowR)
else:    
    RTWater = 'infinite'
    RTFish = 'infinite'
    RTSusp_Particles_In_Water = 'infinite'

if (SedimentFlowR!= 0.0):
    RTSediment = V_Sediment / (24.0 * SedimentFlowR)
else:
    RTSediment = 'infinite'




# Loss Rates (mol.h-1)----------------
LRAdv_Air = Fugacity * DAir_Adv
LRAdv_Water = Fugacity * DWater_Adv
LRAdv_Sediment = Fugacity * DSediment_Adv
LRAdv_SuspParticles = Fugacity * DSuspParticles_Adv
LRAdv_Aerosol = Fugacity * DAerosol_Adv
LRAdv_Fish = Fugacity * DFish_Adv


# Loss Rates (kg.h-1)----------------
LRAdvkg_Air = LRAdv_Air * mm
LRAdvkg_Water = LRAdv_Water * mm
LRAdvkg_Sediment = LRAdv_Sediment * mm
LRAdvkg_SuspParticles = LRAdv_SuspParticles * mm
LRAdvkg_Aerosol = LRAdv_Aerosol * mm
LRAdvkg_Fish = LRAdv_Fish * mm



# Total Loss Rates by Advection (kg.h-1)----------------
Total_LRAdvkg = LRAdvkg_Air + LRAdvkg_Water + LRAdvkg_Sediment + LRAdvkg_SuspParticles + LRAdvkg_Aerosol + LRAdvkg_Fish
 
print('Total Loss Rates by Advection = ', Total_LRAdvkg, ' (kg.h-1)')



***
## Losses by reactive processes:
***
We prefer, when possible, to use a simple first-order kinetic expression for all reactions:

Two equations are proposed here: <br>
$\boxed{dC/dt = –Ck}$     

with C is the concentration of the chemical (mol/m3), t the time and k is the first-order rate constant.     

Integrating from an initial condition of C$_O$ at zero time gives the following equations:     
$\boxed{ln(C/C_O) = –kt}$     

For an inside on half-lives, i.e., the time, τ$_{1/2}$, which is the time required for C to decrease to half of C$_O$.     
This gives:      
$\boxed{ln(C_{(τ_{1/2})}/C_O) = ln (1/2) = –kτ_{1/2}}$    <br>  
<=> $\boxed{τ_{1/2} = k.ln(2)}$

A major advantage of forcing first-order kinetics on all reactions is that, if a chemical is susceptible to several reactions in the same phase, the total rate constant for reaction is the sum of the rate constants of each reactions.     

Unfortunately, there are many situations in which the real reaction rate is not a first-order reaction. Second-order rate reactions occur when the reaction rate is dependent on the concentration of two reactants.

If kinetics are not of the first order, it may be necessary to write the appropriate equations and accept the increased difficulty of solution.


In [ ]:
# Loss Rates (mol.h-1)----------------
LRReac_Air= Fugacity * DAir_Reac
LRReac_Soil = Fugacity * DSoil_Reac
LRReac_Water = Fugacity * DWater_Reac
LRReac_Sediment = Fugacity * DSediment_Reac
LRReac_SuspParticles = Fugacity * DSuspParticles_Reac
LRReac_Aerosol = Fugacity * DAerosol_Reac
LRReac_Fish = Fugacity * DFish_Reac


# Loss Rates (kg.h-1)----------------
LRReackg_Air = LRReac_Air * mm
LRReackg_Soil = LRReac_Soil * mm
LRReackg_Water = LRReac_Water * mm
LRReackg_Sediment = LRReac_Sediment * mm
LRReackg_SuspParticles = LRReac_SuspParticles * mm
LRReackg_Aerosol = LRReac_Aerosol * mm
LRReackg_Fish = LRReac_Fish * mm


# Total Loss Rates by Reactions (kg.h-1)----------------
Total_LRReackg = LRReackg_Air + LRReackg_Soil + LRReackg_Water + LRReackg_Sediment + LRReackg_SuspParticles + LRReackg_Aerosol+ LRReackg_Fish
print('Total Loss Rates by Reactions = ', Total_LRReackg, ' (kg.h-1)')




## Percentage of losses 

In [ ]:
Total_LRkg = Total_LRAdvkg + Total_LRReackg

# Advection
pLAdv_Air   = 100.0 * LRAdvkg_Air / Total_LRkg
pLAdv_Water = 100.0 * LRAdvkg_Water / Total_LRkg
pLAdv_Sediment = 100.0 * LRAdvkg_Sediment / Total_LRkg
pLAdv_SuspParticles = 100.0 * LRAdvkg_SuspParticles / Total_LRkg
pLAdv_Aerosol = 100.0 * LRAdvkg_Aerosol / Total_LRkg
pLAdv_Fish = 100.0 * LRAdvkg_Fish / Total_LRkg

# Reaction
pLReac_Air   = 100.0 * LRReackg_Air / Total_LRkg
pLReac_Soil  = 100.0 * LRReackg_Soil / Total_LRkg
pLReac_Water = 100.0 * LRReackg_Water / Total_LRkg
pLReac_Sediment = 100.0 * LRReackg_Sediment / Total_LRkg
pLReac_SuspParticles = 100.0 * LRReackg_SuspParticles / Total_LRkg
pLReac_Aerosol = 100.0 * LRReackg_Aerosol / Total_LRkg
pLReac_Fish = 100.0 * LRReackg_Fish / Total_LRkg


# Total
pL_Air = pLAdv_Air + pLReac_Air
pL_Water = pLAdv_Water + pLReac_Water
pL_Sediment = pLAdv_Sediment + pLReac_Sediment
pL_SuspParticles = pLAdv_SuspParticles + pLReac_SuspParticles
pL_Aerosol = pLAdv_Aerosol + pLReac_Aerosol
pL_Fish = pLAdv_Fish + pLReac_Fish
pL_Soil = pLReac_Soil


## Total Retention Time

In [ ]:
# Total retention time (days)
if (Total_LRAdvkg != 0.0):
    RT_Total_Adv = 24.0 * total_Mass / Total_LRAdvkg
else:    
    RT_Total_Adv = 'infinite'
    
if (Total_LRReackg != 0.0):
    RT_Total_Reac = 24.0 * total_Mass / Total_LRReackg
else:    
    RT_Total_Reac = 'infinite'
    
if ((Total_LRAdvkg + Total_LRReackg) != 0.0):    
    RT_Total = 24.0 * total_Mass / (Total_LRAdvkg + Total_LRReackg)
else: 
    RT_Total = 'infinite'

print("Retention Time by Advection (days): ", RT_Total_Adv)
print("Retention Time by Reaction (days): ", RT_Total_Reac)
print("Total Retention Time (days): ", RT_Total)

# To test the results, theoretically, we must have 1.0/RT_Total=1.0/RT_Total_Adv +1.0/RT_Total_Reac
if ((Total_LRAdvkg != 0.0) and (Total_LRReackg != 0.0) and (Total_LRAdvkg + Total_LRReackg)!= 0.0 ):
    print("Data Test: ", 1.0/RT_Total, (1.0/RT_Total_Adv +1.0/RT_Total_Reac))


In [ ]:
import sys

#Saved results in a dataframe:     

dataZ = {'Name':   ['Level II', 
                'Level II', 
                'Level II',
                'Level II',
                'Level II',
                'Level II',
                'Level II'                      
               ],
    'Compartments':   ['Air', 
                'Air',
                'Water', 
                'Water',
                'Water',                      
                'Soil',
                'Sediment'            
               ],
        'Specificity':   ['Dissolved in Air', 
                          'Aerosol',
                          'Dissolved in Water', 
                          'Fish',
                          'Suspended Particle',
                          'Soil',
                          'Sediment'                         
               ],         
         
               'ZValue (mol/m3·Pa)': [ZAir, 
                                      ZAerosol,
                                      ZFish,
                                      ZSuspendedParticles,
                                      ZWater,
                                      ZSoil,
                                      ZSediment
                ],
                'VZ (mol/Pa)': [VZAir, 
                                VZAerosol,
                                VZWater,
                                VZFish,
                                VZSuspendedParticles,
                                VZSoil,
                                VZSediment],  
         
                'Concentrations (g/m3)': [CAir, 
                                CAerosol,          
                                CWater,
                                CFish,
                                CSuspendedParticles,
                                CSoil,
                                CSediment],
         
                'Amounts (kg)':[AAir, 
                                AAerosol,
                                AWater,
                                AFish,
                                ASuspendedParticles,
                                ASoil,
                                ASediment],
         
                'Loss by Advection (kg/h)':[LRAdvkg_Air, 
                                LRAdvkg_Aerosol,
                                LRAdvkg_Water,
                                LRAdvkg_Fish,
                                LRAdvkg_SuspParticles,
                                0.0,
                                LRAdvkg_Sediment], 
         
                'Loss by Reaction (kg/h)':[LRReackg_Air, 
                                LRReackg_Aerosol,
                                LRReackg_Water,
                                LRReackg_Fish,
                                LRReackg_SuspParticles,
                                LRReackg_Soil,
                                LRReackg_Sediment], 
         
               'Residence Time by Advection (days)':[RTAir , 
                                RTAerosol_In_Air,
                                RTWater,
                                RTFish,
                                RTSusp_Particles_In_Water,
                                "infinite", #sys.maxsize,
                                RTSediment],
         
               'Half Lives (hours)':[t05_Air , 
                                t05_Aerosol,
                                t05_Water,
                                t05_Fish,
                                t05_SuspParticles,
                                t05_Soil,
                                t05_Sediment],          
 #          
                '% Amounts': [pAir, 
                      pAerosol,
                      pWater,
                      pFish,
                      pSuspendedParticles,
                      pSoil,
                      pSediment],       
         
                '% Losses': [pL_Air, 
                      pL_Aerosol,
                      pL_Water,
                      pL_Fish,
                      pL_SuspParticles,
                      pL_Soil,
                      pL_Sediment]           
        }
dfZ = pd.DataFrame(dataZ)
pd.set_option('display.precision', 2)
#pd.set_option('display.float_format', lambda x: '%5.2f' % x)
#pd.reset_option('display.float_format')
display(dfZ)


***
***
## Plot data: 

In [ ]:
from ipywidgets import widgets


grid = widgets.GridspecLayout(20, 6)



bTitle = widgets.Button(description="Level II: "+Name+" at equilibrium", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = 'white', font_weight='bold', text_color='black',font_size="20"))
grid[0:1, 0:] = bTitle


bTitle2 = widgets.Button(description="Fugacity: {0:e}".format(Fugacity)+" Pa; ", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = 'white', font_weight='bold', text_color='black',font_size="20"))
grid[1:2, 0:] = bTitle2

bAir = widgets.Button(description="Air: Total Mass = {0:6.3f}".format(AAir+AAerosol)+" kg"+
                                   " ;   {0:3.2f}".format(pAir+pAerosol)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = 'lightblue', font_weight='bold', text_color='black',font_size="20")
                     )
grid[2:5, 0:] = bAir

bSoil = widgets.Button(description="Soil: Total Mass = {0:6.3f}".format(ASoil)+" kg"+
                                   " ;   {0:3.2f}".format(pSoil)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = '#c5672c', font_weight='bold', text_color='white',font_size="24")
                      )
grid[5:-1, 0:2] = bSoil

bWater = widgets.Button(description="Water: Total Mass = {0:6.3f}".format(AWater+ASuspendedParticles+AFish)+" kg"+
                                   " ;   {0:3.2f}".format(pWater+pSuspendedParticles+pFish)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = '#4994ec', font_weight='bold', text_color='white',font_size="24")
)
grid[5:16, 2:] = bWater 


bSediment = widgets.Button(description="Sediments: Total Mass = {0:6.3f}".format(ASediment)+" kg"+
                                   " ;   {0:3.2f}".format(pSediment)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = '#dbad3a', font_weight='bold', text_color='white',font_size="24")
)

grid[16:-1, 2:] = bSediment


display(grid)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

values = '% Amounts'
fig1 = px.sunburst(dfZ, title = "Level II Distribution of " + Name + ' ' + values, 
                  path=['Name','Compartments','Specificity'], values=values)
fig1.show()

values = '% Losses'
fig2 = px.sunburst(dfZ, title = "Level II Distribution of " + Name + ' ' + values, 
                  path=['Name','Compartments','Specificity'], values=values)
fig2.show()
                 
values = 'Concentrations (g/m3)'
fig3 = px.sunburst(dfZ, title = "Level II Distribution of " + Name + ' ' + values, 
                  path=['Name','Compartments','Specificity'], values=values)
fig3.show()


     

In [ ]:
values = '% Amounts'
fig4 = px.bar(dfZ, x='Compartments', y=values, color='Specificity', title = "Level II Distribution of " + Name + ' ' + values,text_auto=True)
fig4.show()

values = '% Losses'
fig5 = px.bar(dfZ, x='Compartments', y=values, color='Specificity', title = "Level II Distribution of " + Name + ' ' + values,text_auto=True)
fig5.show()

values = 'Concentrations (g/m3)'
fig5 = px.bar(dfZ, x='Compartments', y=values, color='Specificity', title = "Level II Distribution of " + Name + ' ' + values,text_auto=True)
fig5.show()

In [ ]:
masque = dfZ["Residence Time by Advection (days)"]!= "infinite"

fig6 = px.scatter(dfZ[masque].sort_values(by = 'Residence Time by Advection (days)'),
                   x='Specificity', y="Residence Time by Advection (days)", 
                   size='Concentrations (g/m3)', size_max = 150, color='Compartments', 
                   title = 'Concentrations (g/m3) and Residence Time (days).')
fig6.update_yaxes(type='category', automargin=True)
fig6.show()

In [ ]:
import plotly.graph_objects as go


# Colors definitions: ---------------------------------------------------------------

color_Air      = "#bdd6e2"
color_Water    = "#224fab"
color_Sediment = "#d4ae50"
color_Soil     = "#ae7044"
colorAdvection = "#629c70"
colorReacion   = "#c698f1"

color_LinkAir      = "#c4def1"
color_LinkWater    = "#728ec8"
color_LinkSediment = "#f4d381"
color_LinkSoil     = "#e49a6b"


fig7 = go.Figure(data=[go.Sankey(
    node = dict(pad = 15,thickness = 20,line = dict(color = "black", width = 0.5),
    label = ["Dissolved in Air", "Aerosol", 
             "Dissolved in Water", "Suspended Particles", "Fish",
             "Soil",
             "Sediments",
             "Loss by Advection", "Loss by Reaction"
             
            ],
    color = [color_Air,color_Air,
             color_Water,color_Water,color_Water,
             color_Soil, 
             color_Sediment,
             colorAdvection, colorReacion
            ]
    ),
    link = dict(
      source = [0, 0, 1, 1,
                2, 2, 3, 3, 4, 4,
                5,
                6, 6], # indices correspond to source node wrt to label 
      target = [7, 8, 7, 8,
                7, 8, 7, 8, 7, 8,
                8,
                7, 8,], 
      value = [LRAdvkg_Air, LRReackg_Air, LRAdvkg_Aerosol, LRReackg_Aerosol,
               LRAdvkg_Water, LRReackg_Water,LRAdvkg_SuspParticles, LRReackg_SuspParticles, LRAdvkg_Fish, LRReackg_Fish,
               LRReackg_Soil,
               LRAdvkg_Sediment, LRReackg_Sediment],
      color = [color_LinkAir, color_LinkAir, 
               color_LinkAir, color_LinkAir, 
               color_LinkWater, color_LinkWater,
               color_LinkWater, color_LinkWater,
               color_LinkWater, color_LinkWater, 
               color_LinkSoil, color_LinkSoil,
               color_LinkSediment,color_LinkSediment]
  ))])

fig7.update_layout(
    hovermode = 'x',
    title="Compartments Losses by Advection and reaction (kg.h-1)",
    font=dict(size = 20, color = 'black')
   # paper_bgcolor='#666869'

)

fig7.show()




In [ ]:
fig8 = go.Figure(data=[go.Sankey(
    node = dict(pad = 15,thickness = 20,line = dict(color = "black", width = 0.5),
    label = ["Dissolved in Air", "Aerosol", 
             "Dissolved in Water", "Suspended Particles", "Fish",
             "Soil",
             "Sediments",
             "Loss by Advection"
            ],
    color = [color_Air,color_Air,
             color_Water,color_Water,color_Water,
             color_Soil, 
             color_Sediment,
             '#a6cee3','#fdbf6f'
            ]
    ),
    link = dict(
      source = [0, 1, 
                2, 3, 4,
                6], # indices correspond to source node wrt to label 
      target = [7, 7,
                7, 7, 7,
                7], 
      value = [LRAdvkg_Air, LRAdvkg_Aerosol,
               LRAdvkg_Water,LRAdvkg_SuspParticles,LRAdvkg_Fish,
               LRAdvkg_Sediment],
      color = [color_LinkAir,
               color_LinkAir,
               color_LinkWater,
               color_LinkWater,
               color_LinkWater,
               color_LinkSediment]
  ))])

fig8.update_layout(
    hovermode = 'x',
    title="Compartments Losses by Advection (kg.h-1)",
    font=dict(size = 20, color = 'black'),
)

fig8.show()

In [ ]:
fig9 = go.Figure(data=[go.Sankey(
    node = dict(pad = 15,thickness = 20,line = dict(color = "black", width = 0.5),
    label = ["Dissolved in Air", "Aerosol", 
             "Dissolved in Water", "Suspended Particles", "Fish",
             "Soil",
             "Sediments",
             "Loss by Reaction"
            ],
    color = [color_Air,color_Air,
             color_Water,color_Water,color_Water,
             color_Soil, 
             color_Sediment,
             '#a6cee3','#fdbf6f'
            ]
    ),
    link = dict(
      source = [0, 1, 
                2, 3, 4,
                5,
                6], # indices correspond to source node wrt to label 
      target = [7, 7,
                7, 7, 7,
                7, 
                7], 
      value = [LRReackg_Air, LRReackg_Aerosol,
               LRReackg_Water,LRReackg_SuspParticles,LRReackg_Fish,
               LRReackg_Soil,
               LRReackg_Sediment],
      color = [color_LinkAir,
               color_LinkAir,
               color_LinkWater,
               color_LinkWater,
               color_LinkWater,
               color_LinkSoil,
               color_LinkSediment]
  ))])

fig9.update_layout(
    hovermode = 'x',
    title="Compartments Losses by Reactions (kg.h-1)",
    font=dict(size = 20, color = 'black')
    # , paper_bgcolor='#f7f7f7'
)

fig9.show()

***
## Save results in a html file: 

Plots and results are saved in a html file in the same folder as the actual jupyter file.

In [ ]:
import json

from ipywidgets import IntSlider
from ipywidgets.embed import embed_data


data = embed_data(views=[grid])

html_template = """
<!-- Load RequireJS, used by the IPywidgets for dependency management -->
<script 
    src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" 
    integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" 
    crossorigin="anonymous">
</script>

<!-- Load IPywidgets bundle for embedding. -->
<script
      data-jupyter-widgets-cdn="https://unpkg.com/"
      data-jupyter-widgets-cdn-only
      src="https://cdn.jsdelivr.net/npm/@jupyter-widgets/html-manager@*/dist/embed-amd.js" 
      crossorigin="anonymous">
</script>

<!-- The state of all the widget models on the page -->
<script type="application/vnd.jupyter.widget-state+json">
      {manager_state}
</script>

<div id="first-slider-widget">
<!-- This script tag will be replaced by the view's DOM tree -->
    <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[0]}
    </script>
</div>

"""
manager_state = json.dumps(data['manager_state'])
widget_views = [json.dumps(view) for view in data['view_specs']]
rendered_template = html_template.format(manager_state=manager_state, widget_views=widget_views)



with open("Level2_"+Name+' '+'.html', 'w') as fp:
    fp.write("<h3>Retention Time by Advection (days): {0:3.2f}".format(RT_Total_Adv)+"</h3>")
    fp.write("<h3>Retention Time by Reaction (days): {0:3.2f}".format(RT_Total_Reac)+"</h3>")
    fp.write("<h3>Total Retention Time (days): {0:3.2f}".format(RT_Total)+"</h3>")
    fp.write(dfZ.to_html())
    fp.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(fig3.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(fig4.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(fig5.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(fig6.to_html(full_html=False, include_plotlyjs='cdn'))    
    fp.write(fig7.to_html(full_html=False, include_plotlyjs='cdn'))    
    fp.write(fig8.to_html(full_html=False, include_plotlyjs='cdn'))       
    fp.write(fig9.to_html(full_html=False, include_plotlyjs='cdn'))        
    fp.write(rendered_template)
fp.close()